In [1]:
import os
import re
import pandas as pd

In [2]:
# Function to filter bad time based on timedelta
def filter_badtime(df):
    df = df.copy()
    # df = df[df['Interaction Time'] > pd.to_timedelta("9:35:00")]  # playground
    df = df[df['Time'] > pd.to_timedelta("9:47:00")]  # toilet
    # df = df[~((df['Interaction Time'] >= pd.to_timedelta("10:20:00")) & (df['Interaction Time'] <= pd.to_timedelta("10:27:00")))]  # toilet
    # df = df[~((df['Interaction Time'] >= pd.to_timedelta("10:33:00")) & (df['Interaction Time'] <= pd.to_timedelta("10:37:00")))]  # toilet
    # df = df[~((df['Interaction Time'] >= pd.to_timedelta("11:30:00")) & (df['Interaction Time'] <= pd.to_timedelta("11:32:00")))]  # toilet
    # df = df[~((df['Interaction Time'] >= pd.to_timedelta("11:38:00")) & (df['Interaction Time'] <= pd.to_timedelta("11:41:00")))]  # toilet
    # df = df[~((df['Interaction Time'] >= pd.to_timedelta("11:45:00")) & (df['Interaction Time'] < pd.to_timedelta("12:02:00")))]  # toilet
    # df = df[~((df['Interaction Time'] >= pd.to_timedelta("12:19:00")) & (df['Interaction Time'] <= pd.to_timedelta("12:26:00")))]  # toilet
    df = df[df['Time'] < pd.to_timedelta("12:27:00")]  # toilet
    return df


In [3]:
# Step 2: Calculate bad time
def calculate_bad_time(row):
    bad_time_ranges = [
    (pd.to_timedelta("08:55:00"), pd.to_timedelta("09:47:00")),
    # (pd.to_timedelta("10:20:00"), pd.to_timedelta("10:27:00")),
    # (pd.to_timedelta("10:33:00"), pd.to_timedelta("10:37:00")),
    # (pd.to_timedelta("11:30:00"), pd.to_timedelta("11:32:00")),
    # (pd.to_timedelta("11:38:00"), pd.to_timedelta("11:41:00")),
    # (pd.to_timedelta("11:45:00"), pd.to_timedelta("12:02:00")),
    # (pd.to_timedelta("12:19:00"), pd.to_timedelta("12:26:00"))
    ]
    
    total_bad_time = pd.Timedelta(0)
    for start, end in bad_time_ranges:
        # Check if the bad time range overlaps with the current range
        overlap_start = max(row['Start_Time'], start)
        overlap_end = min(row['End_Time'], end)
        if overlap_start < overlap_end:
            total_bad_time += overlap_end - overlap_start
    return total_bad_time

In [4]:
# The correct one
map_HL = {    
    '27': 'HL',
    '28': 'HL',
    '29': 'HL',
    '30': 'HL',
    '31': 'HL',
    '32': 'TH',
    '33': 'HL',
    '41': 'TH',
    '42': 'TH',
    '43': 'TH',
    '44': 'TH',
    '45': 'TH',
    '46': 'TH'
}

In [5]:
def read_a_file(chunk_size = 100000, file_path = None):
    # Columns to delete
    columns_to_delete = [' Angle1', ' Angle2', ' Leftx', 'Lefty', 'Rightx', 'Righty',
                        ' Leftx2', 'Lefty2', 'Rightx2', 'Righty2', 
                        ' Gender1', ' Gender2', ' WasTalking1', ' WasTalking2 ']
    
    # Incremental processing
    processed_chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        chunk.drop(columns=columns_to_delete, inplace=True, errors='ignore')
        chunk = chunk[~((chunk['Type1'] == 'TEACHER') | (chunk[' Type2'] == 'TEACHER'))]
        processed_chunks.append(chunk)
        
    processed_df = pd.concat(processed_chunks, ignore_index=True)
    
    processed_df.columns = processed_df.columns.str.strip()
    # Remove leading space
    processed_df.rename(columns={'Person 1': 'Person1'}, inplace=True)
    processed_df.rename(columns={' Interaction Time': 'Interaction Time'}, inplace=True)
    
    processed_df['Person1'] = processed_df['Person1'].str.extract(r'(\d+)$')
    processed_df['Person2'] = processed_df['Person2'].str.extract(r'(\d+)$')
    
    processed_df['Diagnosis1'] = processed_df['Person1'].map(map_HL)
    processed_df['Diagnosis2'] = processed_df['Person2'].map(map_HL)
    
    processed_df['Time'] = pd.to_timedelta(processed_df['Interaction Time'])
    
    processed_df = filter_badtime(processed_df)

    print(processed_df.head())
    return processed_df
    

In [6]:
# 3
# directory_path = r"D:\OneDrive - Delft University of Technology\Thesis\DEBBIE_STARFISH_2223\Version 3_ Kalman_Fixed_Data + Overlap Filter\Synched_Data_GR0_2to2_ANGLE45\PAIRANGLES"
# 2
# directory_path = r"D:\OneDrive - Delft University of Technology\Thesis\DEBBIE_STARFISH_2223\Version 2_Kalman_Fixed_Data\Synched_Data_GR0_2to2_ANGLE45\PAIRANGLES"
# 1
directory_path = r"D:\OneDrive - Delft University of Technology\Thesis\DEBBIE_STARFISH_2223\Synched_Data_GR0_22_DEN\PAIRANGLES"

# File pattern to match
# 3
# file_pattern = re.compile(r'DAILY_ANGLES(\d{6})_GR0_22_DEN_072924_V21588968411.CSV')
# 2
# file_pattern = re.compile(r'DAILY_ANGLES(\d{6})_GR0_22_DEN_072224_V22072780461.CSV')
# 1
file_pattern = re.compile(r'DAILY_ANGLES(\d{6})_GR0_22_DEN_010224_V21201895560.CSV')

# Function to extract date from filename
def extract_date_from_filename(filename):
    match = file_pattern.search(filename)
    if match:
        date_str = match.group(1)
        return pd.to_datetime(date_str, format='%m%d%y').date()
    return None

# List of all files in the directory
files = [f for f in os.listdir(directory_path) if file_pattern.match(f)]
final_results_list = []


for file in files:
    file_path = os.path.join(directory_path, file)
    date = extract_date_from_filename(file)
    processed_df = read_a_file(file_path=file_path, chunk_size = 1000)
    
    # Step 1: Calculate the start and end times for each pair
    start_time_df = processed_df.groupby(['Person1', 'Person2'])['Time'].min().reset_index()
    end_time_df = processed_df.groupby(['Person1', 'Person2'])['Time'].max().reset_index()
    # Merge start and end times
    time_range_df = pd.merge(start_time_df, end_time_df, on=['Person1', 'Person2'])
    time_range_df.columns = ['Person1', 'Person2', 'Start_Time', 'End_Time']
    # Calculate total time
    time_range_df['Total_Time'] = time_range_df['End_Time'] - time_range_df['Start_Time']
    # Apply the function to calculate bad time
    time_range_df['Bad_Time'] = time_range_df.apply(calculate_bad_time, axis=1)
    # Adjust total time by subtracting bad time in milliseconds
    time_range_df['Adjusted_Total_Time'] = time_range_df['Total_Time']  - time_range_df['Bad_Time']
    time_range_df['Adjusted_Total_Time'] = time_range_df['Adjusted_Total_Time'].dt.total_seconds() * 1000

    interaction_df = processed_df[processed_df['45Interaction'] == 0.1]
    # Count occurrences of Interaction = 0.1 for each pair
    interaction_counts = interaction_df.groupby(['Person1', 'Person2']).size().reset_index(name='Count_Interactions')

    # Calculate Accumulated Interaction Time, each is 0.1 second
    interaction_counts['Accumulated_Interaction_Time'] = interaction_counts['Count_Interactions'] * 0.1 * 500

    # Step 4: Merge with adjusted total time
    results_df = pd.merge(interaction_counts, time_range_df[['Person1', 'Person2', 'Adjusted_Total_Time']], on=['Person1', 'Person2'])
    results_df['Social_Contact_Ratio'] = results_df['Accumulated_Interaction_Time'] / results_df['Adjusted_Total_Time']

    results_df['Date'] = date
    
    # reversed_df = results_df.copy()
    # reversed_df = reversed_df.rename(columns={
    #     'Person1': 'Person2',
    #     'Person2': 'Person1',
    # })
    # results_df = pd.concat([results_df, reversed_df], ignore_index=True)

    # results_df.to_csv("reverse.csv", index=False)

    results_df.rename(columns={
        'Person1': 'Subject',
        'Person2': 'Partner',
        'Social_Contact_Ratio': 'scLag'
    }, inplace=True)

    # Extract unique pairs with their diagnosis
    diagnosis_info = processed_df[['Person1', 'Person2', 'Diagnosis1', 'Diagnosis2']].drop_duplicates()
    # Rename columns for clarity
    diagnosis_info.columns = ['Subject', 'Partner', 'diagnosisPerson1', 'diagnosisPerson2']

    # reversed_diagnosis_info = diagnosis_info.copy()
    # reversed_diagnosis_info = reversed_diagnosis_info.rename(columns={
    #     'Subject': 'Partner',
    #     'Partner': 'Subject',
    #     'diagnosisPerson1': 'diagnosisPerson2',
    #     'diagnosisPerson2': 'diagnosisPerson1',
    # })
    # diagnosis_info = pd.concat([diagnosis_info, reversed_diagnosis_info], ignore_index=True)

    # diagnosis_info.to_csv("diagnosis_info.csv", index=False)

    # Step 3: Ensure matching data types for merging columns
    results_df['Subject'] = results_df['Subject'].astype(str)
    results_df['Partner'] = results_df['Partner'].astype(str)
    diagnosis_info['Subject'] = diagnosis_info['Subject'].astype(str)
    diagnosis_info['Partner'] = diagnosis_info['Partner'].astype(str)

    # Merge with results_df to add diagnosis information
    results_df = pd.merge(results_df, diagnosis_info, on=['Subject', 'Partner'], how='left')

    # Determine Pair type: 'Same' if both have the same diagnosis, otherwise 'DD'
    results_df['Pair'] = results_df.apply(lambda row: 'same' if row['diagnosisPerson1'] == row['diagnosisPerson2'] else 'diff', axis=1)

    final_results_list.append(results_df)

# Combine all results into a single DataFrame
final_results_df = pd.concat(final_results_list, ignore_index=True)

# Save or process final_results_df as needed
final_results_df.to_csv('final_social_contact_ratios_with_diagnosis.csv', index=False)
print(final_results_df.head())


       Person1 Person2 Interaction Time  Interaction Millisecond  Interaction  \
858460      27      28       9:47:01 AM                        0          0.1   
858461      27      29       9:47:01 AM                        0          0.0   
858462      27      30       9:47:01 AM                        0          0.0   
858463      27      31       9:47:01 AM                        0          0.0   
858464      27      32       9:47:01 AM                        0          0.0   

        45Interaction  Type1  Type2 Diagnosis1 Diagnosis2            Time  
858460            0.0  CHILD  CHILD         HL         HL 0 days 09:47:01  
858461            0.0  CHILD  CHILD         HL         HL 0 days 09:47:01  
858462            0.0  CHILD  CHILD         HL         HL 0 days 09:47:01  
858463            0.0  CHILD  CHILD         HL         HL 0 days 09:47:01  
858464            0.0  CHILD  CHILD         HL         TH 0 days 09:47:01  
        Person1 Person2 Interaction Time  Interaction Mil

In [ ]:
results_df